## 使用fastText对文本进行分类

jrj info_item 人工来源的资讯
**第一步获取分类文本：输出数据格式： 样本 + 样本标签**

In [1]:
import jieba
import os
import csv

In [42]:
string = "25 家 公司 遭 问询 ： 中国 石化 货币资金 超 1700 亿 还要 募 120 亿 ， 新规后 再融资 “ 遇冷 ”   \n" + "           坐拥"
if "\n" in string :
    print("The string contains a newline character.")
elif "\r\n" in string:
    print("The string contains a newline r character.")
else:
    print("The string does not contain a newline character.")

The string contains a newline character.


In [43]:
from bs4 import BeautifulSoup
import sys
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)
basedir = "D:\\github\\data\\fast_txt\\THUCNews\\THUCNews\\" #这是我的文件地址，需跟据文件夹位置进行更改

##生成fastext的训练和测试数据集
input_csv_file = basedir+"db_info_231111.csv"
ftrain = open(basedir+"news_fasttext_train_info_item.txt","w", encoding='utf-8')
ftest = open(basedir+"news_fasttext_test_info_item.txt","w", encoding='utf-8')

with open(input_csv_file, 'r', newline='', encoding='utf-8') as csv_file:
    reader = csv.reader(csv_file)
    count = 0
    for row in reader:
        count += 1
        title = row[0]
        content = row[1]
        tag = row[2]
        soup = BeautifulSoup(content, 'html.parser')
        pure_text = soup.get_text()
        text = title + "\n" + pure_text
        seg_text = jieba.cut(text.replace("\t"," ").replace("\n"," "))
        outline = " ".join(seg_text)
        outline = outline + "\t__label__" + tag + "\n"
        if count < 10000:
            ftrain.write(outline)
            ftrain.flush()
        elif count  < 20000:
            ftest.write(outline)
            ftest.flush()
        else: break

ftrain.close()
ftest.close()
print('完成输出数据！')



C:\Users\Jack9\AppData\Local\Temp\ipykernel_28560\617880510.py:29: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(content, 'html.parser')


完成输出数据！


## 第二步,使用fastText进行训练模型(如果数据已经准备好，可以直接运行第二步)

In [34]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import fasttext
#训练模型
classifier = fasttext.train_supervised(basedir+"news_fasttext_train_info_item.txt",label_prefix="__label__")

#load训练好的模型
#classifier = fasttext.load_model('news_fasttext.model.bin', label_prefix='__label__')
print('训练完成！')

训练完成！


## 测试模型

In [35]:
result = classifier.test(basedir+"news_fasttext_test_info_item.txt")
print('precision：', result[1])

precision： 0.35732905554441546


### 由于fasttext貌似只提供全部结果的p值和r值，想要统计不同分类的结果，就需要自己写代码来实现了。

In [44]:
labels_right = []
texts = []
with open(basedir+"news_fasttext_test_info_item.txt", encoding='utf-8') as fr:
    for line in fr:
        temp_line = line
        line = str(line.encode("utf-8"), 'utf-8').rstrip()
        lines = line.split("\t")
        print(len(lines))
        if len(lines) > 1:
            labels_right.append(lines[1].replace("__label__",""))
            texts.append(lines[0])
    #     print labels
    #     print texts
#     break
labels_predict = [term[0] for term in classifier.predict(texts)[0]] #预测输出结果为二维形式
# print labels_predict

text_labels = list(set(labels_right))
text_predict_labels = list(set(labels_predict))
print(text_predict_labels)
print(text_labels)
print()

A = dict.fromkeys(text_labels,0)  #预测正确的各个类的数目
B = dict.fromkeys(text_labels,0)   #测试数据集中各个类的数目
C = dict.fromkeys(text_predict_labels,0) #预测结果中各个类的数目
for i in range(0,len(labels_right)):
    B[labels_right[i]] += 1
    C[labels_predict[i]] += 1
    if labels_right[i] == labels_predict[i].replace('__label__', ''):
        A[labels_right[i]] += 1

print('预测正确的各个类的数目:', A) 
print()
print('测试数据集中各个类的数目:', B)
print()
print('预测结果中各个类的数目:', C)
print()
#计算准确率，召回率，F值
for key in B:
    try:
        r = float(A[key]) / float(B[key])
        p = float(A[key]) / float(C['__label__' + key])
        f = p * r * 2 / (p + r)
        print("%s:\t p:%f\t r:%f\t f:%f" % (key,p,r,f))
    except:
        print("error:", key, "right:", A.get(key,0), "real:", B.get(key,0), "predict:",C.get(key,0))

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
2
1
1
1
1
1
1
1
2
2
2
2
2
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
